In [4]:
%pip install stable-baselines3[extra]
%pip install gym --upgrade

Note: you may need to restart the kernel to use updated packages.



In [5]:
##IMPORT DEPENDENCIES##
#gym stuff#
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

#helpers#
import numpy as np
import random
import os

#stablebaselines#
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3. common.evaluation import evaluate_policy

In [6]:
Discrete(3).sample()

0

In [7]:
Box(0,1, shape=(3,)).sample()

array([0.64618695, 0.03272159, 0.6033102 ], dtype=float32)

In [8]:
Tuple((Discrete(3),Box(0,1, shape=(3,)))).sample()

(0, array([0.8100594 , 0.03465777, 0.93658704], dtype=float32))

In [9]:
Dict({'height':Discrete(2), 'speed':Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([19.3767], dtype=float32))])

In [10]:
MultiBinary(4).sample()

array([0, 0, 1, 0], dtype=int8)

In [11]:
MultiDiscrete([5, 2, 2]).sample()


array([4, 1, 0], dtype=int64)

In [12]:
##BUILDING ENVIRONMENT##
#-Build an agent to give the best shower possible
#-Random Temp
#- 37 and 39 degrees

class ShowerEnv(Env):
    def __init__(self):
        self. action_space = Discrete(3) #Up, Down, Hold
        self.observation_space = Box(low=0, high=100, shape=(1,)) #temps between 0 and 100
        self.state = 38 + random.randint(-3,3) #set initial state to 38
        self.shower_length= 60 #length of the shower is 60s
        
    def step(self, action):
        #apply temp adj
        self.state += action-1  

        #decrease shower time
        self.shower_length -= 1

        #Calculate Reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1

        if self.shower_length <=0:
            done = True
        else:
            done = False
        info = {}

        return self.state, reward, done, info
        
    def render(self):
        #implement viz??
        pass
        
    def  reset(self):
        self.state = np.array([38+random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state

In [13]:
env = ShowerEnv()

In [14]:
env.observation_space.sample()

array([47.102383], dtype=float32)

In [15]:
env.action_space.sample()

2

In [16]:
##TEST ENVIRONMENT##
episodes = 5
for episode in range (1, episodes+1):
    obs= env.reset()
    done=False
    score=0

    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'. format(episode, score))
#env.close()    

Episode:1 Score:-44
Episode:2 Score:10
Episode:3 Score:-32
Episode:4 Score:-22
Episode:5 Score:-40


In [17]:
##TRAIN MODEL##
log_path = os.path.join('Training','Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\Diogo\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [18]:
model.learn(total_timesteps=50000)

Logging to Training\Logs\PPO_1


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -34.4    |
| time/              |          |
|    fps             | 588      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -33         |
| time/                   |             |
|    fps                  | 423         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010193411 |
|    clip_fraction        | 0.0539      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.000515   |
|    learning_rate        | 0.

In [19]:
shower_path= os.path.join('Training', 'Saved Models', 'Shower_Model_PPO')

In [20]:
model.save(shower_path)

c:\Users\Diogo\anaconda3\Lib\site-packages\stable_baselines3\common\save_util.py:284: UserWarning: Path 'Training\Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [21]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

c:\Users\Diogo\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
c:\Users\Diogo\anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
c:\Users\Diogo\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(24.0, 54.99090833947008)